In [3]:
import pandas as pd
import numpy as np

In [4]:
url = "https://opendata.rhein-kreis-neuss.de/api/v2/catalog/datasets/rhein-kreis-neuss-ladesaulen-in-deutschland/exports/csv"
df = pd.read_csv(url, delimiter=";")


In [5]:
list(df)

['betreiber',
 'art_der_ladeeinrichung',
 'anzahl_ladepunkte',
 'anschlussleistung',
 'steckertypen1',
 'steckertypen2',
 'steckertypen3',
 'steckertypen4',
 'p1_kw',
 'p2_kw',
 'p3_kw',
 'p4_kw',
 'kreis_kreisfreie_stadt',
 'ort',
 'postleitzahl',
 'strasse',
 'hausnummer',
 'adresszusatz',
 'inbetriebnahmedatum',
 'koordinaten']

In [6]:
#list(df)

In [7]:
from geopy.geocoders import Nominatim
def get_state_by_coord(coord):
    geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.reverse(coord, exactly_one=True)
    address = location.raw['address']
    state = address.get('state', '')
    return state

In [8]:
selected_columns = ["betreiber", "anzahl_ladepunkte", "anschlussleistung", "koordinaten"]
df = df[selected_columns].rename(columns={"betreiber": "operator", "anzahl_ladepunkte": "number_of_charging_point", "anschlussleistung" : "connected_load", 
                                          "koordinaten" : "coordinates"})

In [15]:
#df["coordinates"] = df["coordinates"].map(get_state_by_coord)

df_new = df.iloc[:10]
df_new["coordinates"] = df_new["coordinates"].map(get_state_by_coord, na_action='ignore')
df_new.rename(columns={"coordinates" : "State"}, inplace=True)
df_new.to_csv("./mobi-data-tmp.csv", index=False)
df_new

/var/folders/jt/lp68flw167df0vb5tj2dgvfc0000gn/T/ipykernel_8253/2955718872.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["coordinates"] = df_new["coordinates"].map(get_state_by_coord, na_action='ignore')
/var/folders/jt/lp68flw167df0vb5tj2dgvfc0000gn/T/ipykernel_8253/2955718872.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.rename(columns={"coordinates" : "State"}, inplace=True)


,operator,number_of_charging_point,connected_load,State
0,GGEW AG,1,4.6,Hessen
1,Pfalzwerke AG,2,75.0,Hessen
2,ENTEGA Energie GmbH,2,44.0,Hessen
3,Entega Plus GmbH,2,22.0,Hessen
4,Entega Plus GmbH,2,22.0,Hessen
5,Vogel Autohäuser GmbH & Co. KG,2,44.0,Hessen
6,Entega Plus GmbH,2,150.0,Hessen
7,Entega Plus GmbH,2,22.0,Hessen
8,ALDI SÜD,2,44.0,Hessen
9,ENTEGA Energie GmbH,2,44.0,Hessen


In [14]:
import os.path as path
file_path = "./mobi-data-tmp.csv"
if not path.isfile(file_path):
    print("Yes it present")
else:
    print("No it is not")

Yes it present


In [16]:
df_new = df_new.drop(columns=["operator"])
df_new.head()

,number_of_charging_point,connected_load,State
0,1,4.6,Hessen
1,2,75.0,Hessen
2,2,44.0,Hessen
3,2,22.0,Hessen
4,2,22.0,Hessen


In [17]:
df_new = df_new.groupby(["State"]).agg({"number_of_charging_point": sum, "connected_load": sum}).reset_index()
df_new

,State,number_of_charging_point,connected_load
0,Hessen,19,471.6


In [18]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../tmp.sqlite', echo=False)
df_new.to_sql("mobi", con=engine, if_exists="replace", index=False)

1

In [ ]:
db = pd.read_sql_table("mobi", "sqlite:///tmp.sqlite")
len(db.index)

2

In [ ]:
df = pd.read_excel("fz28_2023_04.xlsx;jsessionid=3FA0B971D18E156CCB2B0AC11B4C40BF.xlsx", sheet_name="FZ 28.9", usecols="B:C")
df.rename(columns={"Unnamed: 1": "State", "Unnamed: 2": "EV Sales"}, inplace=True)
df = df.iloc[12:28]
df

,State,EV Sales
12,Baden-Württemberg,29256
13,Bayern,43821
14,Berlin,6367
15,Brandenburg,3349
16,Bremen,1349
17,Hamburg,5907
18,Hessen,25011
19,Mecklenburg-Vorpommern,2150
20,Niedersachsen,19412
21,Nordrhein-Westfalen,39416


In [ ]:
# ["Unnamed: 1", "Unnamed: 2"]
df1 = pd.read_excel("https://www.kba.de/SharedDocs/Downloads/DE/Statistik/Fahrzeuge/FZ28/fz28_2022_10.xlsx?__blob=publicationFile&v=4", sheet_name="FZ 28.9", usecols="B:C")
df1.rename(columns={"Unnamed: 1": "State", "Unnamed: 2": "EV Sales"}, inplace=True)
df1 = pd.DataFrame(df1.iloc[12:28])
df1.reset_index()
df1

,State,EV Sales
12,Baden-Württemberg,29845
13,Bayern,42870
14,Berlin,6050
15,Brandenburg,3692
16,Bremen,1168
17,Hamburg,6598
18,Hessen,22815
19,Mecklenburg-Vorpommern,1930
20,Niedersachsen,20114
21,Nordrhein-Westfalen,44174


In [ ]:
for ind in df1.index:
    index = df.index[df1["State"][ind]==df["State"]].to_list()[0]
    df["EV Sales"][index] += df1["EV Sales"][ind]
    
df

,State,EV Sales
12,Baden-Württemberg,59101
13,Bayern,86691
14,Berlin,12417
15,Brandenburg,7041
16,Bremen,2517
17,Hamburg,12505
18,Hessen,47826
19,Mecklenburg-Vorpommern,4080
20,Niedersachsen,39526
21,Nordrhein-Westfalen,83590


In [ ]:
s = "betreiber;art_der_ladeeinrichung;anzahl_ladepunkte;anschlussleistung;steckertypen1;steckertypen2;steckertypen3;steckertypen4;p1_kw;p2_kw;p3_kw;p4_kw;kreis_kreisfreie_stadt;ort;postleitzahl;strasse;hausnummer;adresszusatz;inbetriebnahmedatum;koordinaten"
s.split(";")

['betreiber',
 'art_der_ladeeinrichung',
 'anzahl_ladepunkte',
 'anschlussleistung',
 'steckertypen1',
 'steckertypen2',
 'steckertypen3',
 'steckertypen4',
 'p1_kw',
 'p2_kw',
 'p3_kw',
 'p4_kw',
 'kreis_kreisfreie_stadt',
 'ort',
 'postleitzahl',
 'strasse',
 'hausnummer',
 'adresszusatz',
 'inbetriebnahmedatum',
 'koordinaten']